# **Heurísticas de Coloración de Grafos**

Implementación de múltiples heurísticas para el problema de coloración de grafos.

In [ ]:
%run librerias.ipynb

In [ ]:
%run data_loader.ipynb

## **1. IMPLEMENTACIÓN DEL GREEDY BASE (First-Fit)**

In [ ]:
def greedy_coloring(G, order):
    """
    Algoritmo greedy: asigna el menor color disponible.

    Args:
        G: grafo NetworkX
        order: lista con orden de procesamiento de nodos

    Returns:
        dict: {nodo: color}
    """
    coloring = {}
    for node in order:
        neighbor_colors = {coloring[v] for v in G.neighbors(node) if v in coloring}

        color = 0
        while color in neighbor_colors:
            color += 1
        coloring[node] = color

    return coloring

## **2. ORDENAMIENTOS HEURÍSTICOS**

In [ ]:
def ordenamiento_aleatorio(G):
    """Baseline: orden aleatorio"""
    nodes = list(G.nodes())
    random.shuffle(nodes)
    return nodes

def ordenamiento_grado_desc(G):
    """Largest Degree First"""
    return sorted(G.nodes(), key=lambda v: G.degree(v), reverse=True)

def ordenamiento_welsh_powell(G):
    """Welsh-Powell: grado desc + desempate por ID"""
    return sorted(G.nodes(), key=lambda v: (G.degree(v), -v), reverse=True)

## **3. DSATUR (heurística dinámica)**

In [ ]:
def dsatur_coloring(G):
    """
    DSATUR: selecciona nodos por saturación dinámica

    Saturación = número de colores distintos en vecinos
    """
    coloring = {}
    uncolored = set(G.nodes())

    while uncolored:
        saturation = {}
        for v in uncolored:
            neighbor_colors = {coloring[u] for u in G.neighbors(v) if u in coloring}
            saturation[v] = len(neighbor_colors)

        next_node = max(uncolored, key=lambda v: (saturation[v], G.degree(v)))

        neighbor_colors = {coloring[u] for u in G.neighbors(next_node) if u in coloring}
        color = 0
        while color in neighbor_colors:
            color += 1

        coloring[next_node] = color
        uncolored.remove(next_node)

    return coloring

## **4. VERSIÓN PARALELA (simulada)**

In [ ]:
def parallel_greedy(G, batch_size=5):
    """
    Simula paralelismo:
    - Procesa nodos en lotes
    - Resuelve conflictos post-proceso
    """
    nodes = list(G.nodes())
    random.shuffle(nodes)

    coloring = {}

    for i in range(0, len(nodes), batch_size):
        batch = nodes[i:i+batch_size]

        for v in batch:
            neighbor_colors = {coloring[u] for u in G.neighbors(v) if u in coloring}
            color = 0
            while color in neighbor_colors:
                color += 1
            coloring[v] = color

        conflicts = [(u, v) for u, v in G.edges()
                     if u in coloring and v in coloring and coloring[u] == coloring[v]]

        for u, v in conflicts:
            if G.degree(u) >= G.degree(v):
                recolor_node = u
            else:
                recolor_node = v

            neighbor_colors = {coloring[n] for n in G.neighbors(recolor_node)}
            color = 0
            while color in neighbor_colors:
                color += 1
            coloring[recolor_node] = color

    return coloring

## **5. MÉTRICAS**

In [ ]:
def evaluar_coloracion(G, coloring):
    """
    Calcula:
    - Número de colores usados
    - Validez (no hay conflictos)
    """
    num_colores = max(coloring.values()) + 1

    conflictos = 0
    for u, v in G.edges():
        if coloring[u] == coloring[v]:
            conflictos += 1

    valido = (conflictos == 0)

    return {
        "num_colores": num_colores,
        "valido": valido,
        "conflictos": conflictos
    }

## **6. EXPERIMENTOS CON MÚLTIPLES EJECUCIONES**

In [ ]:
def experimento_multiple(G, metodo, nombre, repeticiones=10):
    """
    Ejecuta un método múltiples veces y calcula estadísticas
    """
    resultados = []
    tiempos = []

    for _ in range(repeticiones):
        start = time.time()

        if metodo == "dsatur":
            coloring = dsatur_coloring(G)
        elif metodo == "parallel":
            coloring = parallel_greedy(G)
        else:
            order = metodo(G)
            coloring = greedy_coloring(G, order)

        elapsed = time.time() - start
        metrics = evaluar_coloracion(G, coloring)

        resultados.append(metrics["num_colores"])
        tiempos.append(elapsed)

    return {
        "metodo": nombre,
        "colores_promedio": np.mean(resultados),
        "colores_std": np.std(resultados),
        "colores_min": np.min(resultados),
        "tiempo_promedio": np.mean(tiempos),
        "tiempo_std": np.std(tiempos)
    }

## **7. EJECUCIÓN COMPLETA**

In [ ]:
resultados_heuristicas = []

resultados_heuristicas.append(
    experimento_multiple(G, ordenamiento_aleatorio, "Random", repeticiones=10)
)
resultados_heuristicas.append(
    experimento_multiple(G, ordenamiento_grado_desc, "Grado Desc", repeticiones=10)
)
resultados_heuristicas.append(
    experimento_multiple(G, ordenamiento_welsh_powell, "Welsh-Powell", repeticiones=10)
)
resultados_heuristicas.append(
    experimento_multiple(G, "dsatur", "DSATUR", repeticiones=10)
)
resultados_heuristicas.append(
    experimento_multiple(G, "parallel", "Parallel Greedy", repeticiones=10)
)

## **8. VISUALIZACIÓN DE RESULTADOS**

In [ ]:
df_resultados = pd.DataFrame(resultados_heuristicas)
print("\n" + "="*60)
print("RESULTADOS DE HEURÍSTICAS")
print("="*60)
print(df_resultados.to_string(index=False))
print()
print(f"Resultados del grafo : {stats['nodos']} nodos, {stats['aristas']} aristas")
print(f"Densidad: {stats['densidad']}, Grado máximo: {stats['grado_max']}")